In [28]:
import numpy as np
import pandas as pd
import sympy
from ast import literal_eval
import os, time

# Random Functions

In [29]:
RESULT_DIR = "final_results"
benchmark = "rational_2"

In [30]:
df = {}
functions = None
for f_name in os.listdir(RESULT_DIR):
    try:
        benchmark_name = None
        if len(f_name.split("_")) == 4:
            benchmark_name = f_name.split("_")[1]
        elif len(f_name.split("_")) == 5:
            benchmark_name = "_".join(f_name.split("_")[1:-2])
                
        tool_name =  f_name.split("_")[-2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
            
            if tool_name == 'volesti':
                try:
                    df['volesti']['details'] = df['volesti']['details'].apply(literal_eval)
                                        
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['output'] for i in x]))
                    df['volesti']['output2'] = list(zip(df['volesti']['np'].apply(np.mean), df['volesti']['np'].apply(np.std)))            
                
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['time'] for i in x]))
                    df['volesti']['time'] = df['volesti']['np'].apply(np.mean)            
                except:
                    pass           
                 
            if tool_name == 'faza':
                # try:
                df['faza']['details'] = df['faza']['details'].apply(literal_eval)
                df['faza']['n_hrects'] = df['faza']['details'].apply(lambda x: x['logs'][0]['hrect_checked_num'] if 'logs' in x else x['output'][-1]['hrect_checked_num'])
                # except:
                #     pass

    except Exception as e:
        print(e)

list index out of range


In [31]:
tools = sorted(list(df.keys()))
tool0 = tools[0]
tool1 = tools[1]

df_res = df[tool0].join(df[tool1], on='index', lsuffix=f'_{tool0}', rsuffix=f'_{tool1}', how='outer')

for t in tools[2:]:
    df_res = df_res.join(df[t].add_suffix(f'_{t}'), on='index', how='outer')


In [32]:
df_res.columns

Index(['index', 'bechmark_faza', 'formula_faza', 'index_faza', 'output_faza',
       'error_faza', 'time_faza', 'details_faza', 'n_hrects', 'bechmark_gubpi',
       'formula_gubpi', 'bounds', 'index_gubpi', 'output_gubpi', 'error_gubpi',
       'time_gubpi', 'details_gubpi', 'bechmark_latte', 'formula_latte',
       'bounds_latte', 'index_latte', 'output_latte', 'error_latte',
       'time_latte', 'details_latte', 'bechmark_volesti', 'formula_volesti',
       'bounds_volesti', 'index_volesti', 'output_volesti', 'error_volesti',
       'time_volesti', 'details_volesti', 'np_volesti', 'output2_volesti'],
      dtype='object')

In [33]:
df_res = df_res.sort_values('index_gubpi')
selected_cols = [f'formula_gubpi']
selected_cols.extend(['bounds'])
# selected_cols.extend([f'bounds_{t}' for t in tools])


for t in tools:
    selected_cols.append(f"output_{t}")
    df_res[f"output_{t}"] = df_res[f"output_{t}"].fillna(df_res[f"error_{t}"]).fillna('TBD').astype(str).str[:1000]
    selected_cols.append(f"time_{t}")
    df_res[f"time_{t}"] = df_res[f"time_{t}"].fillna(0).apply(lambda x:round(x, 3))
    if t == 'faza': 
        selected_cols.append(f"n_hrects")
    elif t == 'volesti':
        selected_cols.append(f"output2_volesti")
        
    
df_res = df_res[selected_cols]
df_res['formula_gubpi'] = df_res['formula_gubpi'].apply(sympy.parse_expr)
df_res.head()

,formula_gubpi,bounds,output_faza,time_faza,n_hrects,output_gubpi,time_gubpi,output_latte,time_latte,output_volesti,time_volesti,output2_volesti
0,(1.03*x**2 + 0.48*x*y + 8.88*x + 1.1*y**2 + 1....,"[[0.1, 1], [0.1, 1]]","(0.5898230890246405, 0.6896906251302616)",272.643,1284,"(0.711266057983094, 0.8710624008180216)",0.766,Not a monomial: (5381801554707743/4 + (2091359...,0.050,"(0.583847, 0.696218)",0.070,"(0.648784, 0.03218630839036996)"
1,(6.82*x**2 + 1.32*x*y + 8.69*x + 5.6*y**2 + 8....,"[[0.1, 1], [0.1, 1]]","(1.7129047384964033, 1.812863288989478)",589.094,5336,"(1.5639518246240296, 2.8058999441535044)",0.756,Not a monomial: (2488238794122199/8 + (1151232...,0.041,"(1.59775, 1.90326)",0.070,"(1.744203, 0.08964754620735582)"
2,(5.72*x**2 + 7.83*x*y + 4.6*x + 3.43*y**2 + 8....,"[[0.1, 1], [0.1, 1]]","(0.7618363752627153, 0.8617597761952512)",249.278,847,"(0.7126014688609542, 1.305862755213216)",0.799,Not a monomial: (9457559217478042.0 + (5944751...,0.039,"(0.745151, 0.881956)",0.065,"(0.8220304, 0.052084158826268864)"
3,(5.79*x**2 + 2.65*x*y + 4.62*x + 1.86*y**2 + 1...,"[[0.1, 1], [0.1, 1]]","(0.3981118938485614, 0.4980876414244374)",266.304,1061,"(0.5084030815642594, 0.6028950703253301)",0.627,Not a monomial: (8219069319951155/2 + (3670433...,0.043,"(0.417201, 0.507786)",0.063,"(0.4513803, 0.025515351967198098)"
4,(7.82*x**2 + 7.55*x*y + 3.35*x + 5.83*y**2 + 8...,"[[0.1, 1], [0.1, 1]]","(0.7253642810024395, 0.825341176037968)",486.233,3929,"(0.8289588063133213, 1.0884849667935454)",0.637,Not a monomial: (21910012187157464.0 + (776870...,0.043,"(0.711315, 0.850987)",0.061,"(0.7749078, 0.039051415861143884)"


In [34]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)